# issue 230: spikes in PJM caused by broken filter 

process: 

* verify that shaped_profiles shows bug 

It does not! profile_method correctly selects 930_profile during the issue, and `profile` does not show the dips. So the issue seems to be with the application of the profiles. 
Shaped 923 data also does not show issue (though it does show variable nuclear profile, which is still wrong -- that's an issue introduced by `gridemissions`)

* replicate running `impute_hourly_profiles.calculate_hourly_profiles` to reproduce bug 
* fix bug 



In [ ]:
import pandas as pd
import plotly.express as px

In [1]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import oge.impute_hourly_profiles as impute_hourly_profiles
from oge.filepaths import outputs_folder, results_folder
from oge.column_checks import get_dtypes

In [ ]:
problem_profiles = pd.read_csv(
    f"{outputs_folder()}/2020/hourly_profiles_2020.csv", dtype=get_dtypes()
)

In [ ]:
problem_profiles = problem_profiles[
    (problem_profiles.ba_code == "PJM") & (problem_profiles.fuel_category == "nuclear")
]

In [ ]:
problem_profiles.head()

px.line(
    problem_profiles,
    x="datetime_utc",
    y=[
        "residual_profile",
        "scaled_residual_profile",
        "shifted_residual_profile",
        "profile",
    ],
)

In [ ]:
problem_profiles.profile_method.unique()

In [ ]:
problem_profiles[
    (problem_profiles.datetime_utc > "2020-04-15")
    & (problem_profiles.datetime_utc < "2020-04-16")
]

In [ ]:
### Look for the issue in the next output
shaped = pd.read_csv(
    f"{outputs_folder()}/2020/shaped_eia923_data_2020.csv", dtype=get_dtypes()
)

In [ ]:
shaped = shaped[(shaped.ba_code == "PJM")]

In [ ]:
px.line(
    shaped,
    x="datetime_utc",
    y="net_generation_mwh",
    line_group="fuel_category",
    color="fuel_category",
)

In [ ]:
#### Ok, issue is not in 923 shaped data, so it must just be in CEMS data
plant_level = pd.read_csv(
    f"{results_folder()}/2020/plant_data/hourly/us_units/individual_plant_data.csv"
)

In [ ]:
problem_plant = plant_level[plant_level.plant_id_eia == 2410]

In [ ]:
px.line(problem_plant, x="datetime_utc", y="net_generation_mwh")